In [ ]:
# Étape 1 : Installation des dépendances
!pip install plotly
!pip install IPython
!pip install pydub
!pip install boto3
!pip install awscli

# Étape 2 : Importation des bibliothèques
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import time
from IPython.display import display, Audio
from pydub import AudioSegment
import logging
import unittest
import boto3
import json
import os

# Configuration du logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Étape 3 : Fonction d'authentification
def authenticate(username, password):
    """Authentifie l'utilisateur avec un nom d'utilisateur et un mot de passe"""
    if username == "Morrocosysteme" and password == "123456789":
        logging.info("Authentification réussie")
        return True
    else:
        logging.error("Échec de l'authentification")
        return False

# Étape 4 : Service Cloud (Version simplifiée pour Google Colab)
class CloudService:
    def __init__(self):
        # Simulation des services cloud sans dépendances strictes
        pass

    def create_data_bucket(self, bucket_name='iot-sensor-data'):
        """Simuler la création d'un bucket S3"""
        print(f"Bucket {bucket_name} simulé")
        return bucket_name

    def upload_sensor_data(self, data, bucket_name='iot-sensor-data'):
        """Simuler l'upload de données"""
        try:
            # Convertir les données en JSON
            data_json = data.to_json(orient='records')

            # Générer un nom de fichier unique basé sur le timestamp
            filename = f"sensor_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

            # Simulation de l'upload
            print(f"Données uploadées simulées : {filename}")

            # Optionnel : Sauvegarder localement pour vérification
            with open(filename, 'w') as f:
                f.write(data_json)

            return filename
        except Exception as e:
            print(f"Erreur de simulation d'upload : {e}")
            return None

    def create_cloudwatch_alarm(self, metric_name, threshold):
        """Simuler la création d'une alarme CloudWatch"""
        try:
            print(f"Alarme CloudWatch simulée pour {metric_name}")
            return {
                'AlarmName': f'IoT_Sensor_{metric_name}_Anomaly',
                'Threshold': threshold
            }
        except Exception as e:
            print(f"Erreur lors de la simulation de l'alarme : {e}")
            return None

    def publish_metrics(self, sensor_data):
        """Simuler la publication de métriques"""
        try:
            print("Métriques publiées (simulation)")
            # Afficher quelques métriques pour vérification
            print("Dernières valeurs :")
            print(f"Température : {sensor_data['temperature'].iloc[-1]}")
            print(f"Vibration : {sensor_data['vibration'].iloc[-1]}")
            print(f"Pression : {sensor_data['pressure'].iloc[-1]}")
        except Exception as e:
            print(f"Erreur lors de la simulation des métriques : {e}")



# Étape 5 : Classe Simulateur de Capteurs IoT
class IoTSensor:
    def __init__(self, sensor_id, user):
        self.sensor_id = sensor_id
        self.data = []  # Utiliser une liste pour accumuler les données
        self.user = user
        self.cloud_service = CloudService()
        self.bucket_name = self.cloud_service.create_data_bucket()
        self.authenticate_user()

    def authenticate_user(self):
        """Authentifie l'utilisateur"""
        if self.user not in ["admin", "user1", "user2"]:
            raise ValueError("Utilisateur non autorisé")
        logging.info(f"Utilisateur {self.user} authentifié pour le capteur {self.sensor_id}")

    def generate_data(self):
        """Génère des données simulées de capteur"""
        logging.debug(f"Génération de données pour le capteur {self.sensor_id}")
        return {
            "sensor_id": self.sensor_id,
            "timestamp": datetime.now(),
            "temperature": round(np.random.uniform(20.0, 35.0), 2),
            "vibration": round(np.random.uniform(0.1, 5.0), 2),
            "pressure": round(np.random.uniform(980, 1020), 2)
        }

    def collect_data(self, duration_seconds=60):
        """Collecte des données pendant une durée spécifiée"""
        logging.info(f"Début de la collecte de données pour le capteur {self.sensor_id} pendant {duration_seconds} secondes")
        start_time = datetime.now()
        while (datetime.now() - start_time).seconds < duration_seconds:
            data = self.generate_data()
            self.data.append(data)  # Ajouter les données à la liste
            time.sleep(1)  # Attendre 1 seconde entre chaque mesure
            self.plot_realtime()  # Afficher le graphique en temps réel

        # Convertir la liste en DataFrame
        self.data = pd.DataFrame(self.data)

        # Upload des données dans S3
        if self.bucket_name:
            self.cloud_service.upload_sensor_data(self.data, self.bucket_name)

        # Publier les métriques dans CloudWatch
        self.cloud_service.publish_metrics(self.data)

        # Créer des alarmes CloudWatch
        self.cloud_service.create_cloudwatch_alarm('Temperature', 30.0)
        self.cloud_service.create_cloudwatch_alarm('Vibration', 4.0)
        self.cloud_service.create_cloudwatch_alarm('Pressure', 1010)

        logging.info(f"Fin de la collecte de données pour le capteur {self.sensor_id}")

    def plot_realtime(self):
        """Affiche les graphiques en temps réel"""
        logging.debug(f"Mise à jour du graphique en temps réel pour le capteur {self.sensor_id}")
        fig = go.Figure()

        # Ajouter les lignes pour chaque mesure
        fig.add_trace(go.Scatter(x=[d['timestamp'] for d in self.data], y=[d['temperature'] for d in self.data],
                                name='Température', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=[d['timestamp'] for d in self.data], y=[d['vibration'] for d in self.data],
                                name='Vibration', line=dict(color='red')))
        fig.add_trace(go.Scatter(x=[d['timestamp'] for d in self.data], y=[d['pressure'] for d in self.data],
                                name='Pression', line=dict(color='green')))

        # Ajouter des indicateurs dynamiques
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=self.data[-1]['temperature'] if self.data else 0,
            title={'text': "Température"},
            gauge={'axis': {'range': [20, 35]},
                   'bar': {'color': "blue"},
                   'steps': [
                       {'range': [20, 30], 'color': "lightgray"},
                       {'range': [30, 35], 'color': "red"}
                   ]}))

        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=self.data[-1]['vibration'] if self.data else 0,
            title={'text': "Vibration"},
            gauge={'axis': {'range': [0, 5]},
                   'bar': {'color': "red"},
                   'steps': [
                       {'range': [0, 4], 'color': "lightgray"},
                       {'range': [4, 5], 'color': "red"}
                   ]}))

        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=self.data[-1]['pressure'] if self.data else 0,
            title={'text': "Pression"},
            gauge={'axis': {'range': [980, 1020]},
                   'bar': {'color': "green"},
                   'steps': [
                       {'range': [980, 1010], 'color': "lightgray"},
                       {'range': [1010, 1020], 'color': "red"}
                   ]}))

        fig.update_layout(title=f'Données en temps réel du capteur {self.sensor_id}',
                         xaxis_title='Temps',
                         yaxis_title='Valeurs')

        fig.show()

# Étape 6 : Analyse et Détection d'Anomalies
class IoTAnalytics:
    def __init__(self, data):
        self.data = data

    def detect_anomalies(self):
        """Détecte les anomalies basées sur des seuils"""
        logging.debug("Début de la détection des anomalies")
        anomalies = pd.DataFrame()

        # Définir les seuils
        TEMP_THRESHOLD = 30.0
        VIBRATION_THRESHOLD = 4.0
        PRESSURE_THRESHOLD = 1010

        # Détecter les anomalies
        temp_anomalies = self.data[self.data['temperature'] > TEMP_THRESHOLD]
        vib_anomalies = self.data[self.data['vibration'] > VIBRATION_THRESHOLD]
        press_anomalies = self.data[self.data['pressure'] > PRESSURE_THRESHOLD]

        logging.debug("Fin de la détection des anomalies")
        return {
            'temperature': temp_anomalies,
            'vibration': vib_anomalies,
            'pressure': press_anomalies
        }

    def generate_report(self):
        """Génère un rapport statistique"""
        logging.debug("Génération du rapport statistique")
        stats = {
            'temperature': {
                'mean': self.data['temperature'].mean(),
                'max': self.data['temperature'].max(),
                'min': self.data['temperature'].min()
            },
            'vibration': {
                'mean': self.data['vibration'].mean(),
                'max': self.data['vibration'].max(),
                'min': self.data['vibration'].min()
            },
            'pressure': {
                'mean': self.data['pressure'].mean(),
                'max': self.data['pressure'].max(),
                'min': self.data['pressure'].min()
            }
        }
        logging.debug("Rapport statistique généré")
        return stats

# Étape 7 : Classe de gestion d'alerte
class IoTAlert:
    def __init__(self):
        pass

    def trigger_alert(self):
        """Déclenche une alerte sonore pendant 1 minute"""
        logging.debug("Déclenchement de l'alerte")
        # Note : Pour Google Colab, vous devrez uploader manuellement un fichier MP4 d'alarme
        try:
            video_path = '/content/nuclear evacuation alarm.mp4'  # Chemin du fichier d'alarme

            # Vérifier si le fichier existe
            if not os.path.exists(video_path):
                print("Fichier d'alarme non trouvé. Veuillez uploader un fichier MP4 d'alarme.")
                return False

            # Extraire l'audio de la vidéo .mp4
            audio = AudioSegment.from_file(video_path, format="mp4")

            # Limiter la durée de l'audio à 60 secondes (1 minute)
            audio = audio[:60000]  # Découpe de l'audio en ne gardant que les 60 premières secondes

            # Sauvegarder l'audio extrait en format mp3
            audio.export("/content/evacuation_alarm.mp3", format="mp3")

            # Jouer l'audio extrait
            display(Audio("/content/evacuation_alarm.mp3", autoplay=True))

            print("\nALERTE ! L'anomalie a été détectée. L'alarme se déclenche...")

            # Attente de 60 secondes pendant que l'alarme continue
            time.sleep(60)
            print("\nLa période d'alarme est terminée.")
            logging.debug("Fin de l'alerte")
            return True
        except Exception as e:
            print(f"Erreur lors du déclenchement de l'alerte : {e}")
            return False

# Étape 8 : Tests unitaires
class TestIoTSensor(unittest.TestCase):
    def test_generate_data(self):
        sensor = IoTSensor("SENSOR_1", "admin")
        data = sensor.generate_data()
        self.assertIn("sensor_id", data)
        self.assertIn("timestamp", data)
        self.assertIn("temperature", data)
        self.assertIn("vibration", data)
        self.assertIn("pressure", data)

    def test_authenticate_user(self):
        with self.assertRaises(ValueError):
            IoTSensor("SENSOR_1", "unauthorized_user")

    def test_collect_data(self):
        sensor = IoTSensor("SENSOR_1", "admin")
        sensor.collect_data(duration_seconds=2)
        self.assertGreater(len(sensor.data), 0)

    def test_detect_anomalies(self):
        data = pd.DataFrame({
            'timestamp': [datetime.now()],
            'temperature': [31.0],
            'vibration': [4.5],
            'pressure': [1015]
        })
        analytics = IoTAnalytics(data)
        anomalies = analytics.detect_anomalies()
        self.assertFalse(anomalies['temperature'].empty)
        self.assertFalse(anomalies['vibration'].empty)
        self.assertFalse(anomalies['pressure'].empty)

    def test_generate_report(self):
        data = pd.DataFrame({
            'timestamp': [datetime.now()],
            'temperature': [31.0],
            'vibration': [4.5],
            'pressure': [1015]
        })
        analytics = IoTAnalytics(data)
        report = analytics.generate_report()
        self.assertIn('temperature', report)
        self.assertIn('vibration', report)
        self.assertIn('pressure', report)

# Étape 9 : Fonction pour exécuter les tests unitaires
def run_tests():
    suite = unittest.TestLoader().loadTestsFromTestCase(TestIoTSensor)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    return result

# Étape 10 : Fonction principale
def main():
    try:
        # Téléchargement préalable requis : Uploader un fichier MP4 d'alarme dans /content/
        print("IMPORTANT : Assurez-vous d'avoir uploadé un fichier 'nuclear_evacuation_alarm.mp4' dans /content/")

        # Authentification
        username = input("Veuillez entrer votre nom d'utilisateur : ")
        password = input("Veuillez entrer votre mot de passe : ")

        if authenticate(username, password):
            # Créer un capteur IoT avec l'intégration cloud
            sensor = IoTSensor("SENSOR_1", "admin")

            # Collecter des données
            print("Démarrage de la collecte de données pour 60 secondes...")
            sensor.collect_data(60)

            # Analyse des données
            analytics = IoTAnalytics(sensor.data)
            anomalies = analytics.detect_anomalies()

            # Afficher les anomalies
            anomalies_detected = False
            for metric, anomaly_data in anomalies.items():
                if not anomaly_data.empty:
                    print(f"\n{metric.capitalize()} anomalies:")
                    print(anomaly_data)
                    anomalies_detected = True

            # Générer le rapport statistique
            stats = analytics.generate_report()
            print("\nRapport statistique :")
            for metric, stat in stats.items():
                print(f"\n{metric.capitalize()}:")
                for key, value in stat.items():
                    print(f"  {key}: {value:.2f}")

            # Déclencher l'alerte en cas d'anomalies
            if anomalies_detected:
                alert = IoTAlert()
                alert.trigger_alert()

            # Exécuter les tests unitaires
            print("\n--- Exécution des Tests Unitaires ---")
            test_results = run_tests()

            # Vérifier les résultats des tests
            if test_results.wasSuccessful():
                print("\nTous les tests ont réussi !")
            else:
                print("\nCertains tests ont échoué.")

        else:
            print("Authentification échouée. Arrêt du programme.")

    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Étape 11 : Point d'entrée du programme
if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have

Données uploadées simulées : sensor_data_20241127_194352.json
Métriques publiées (simulation)
Dernières valeurs :
Température : 26.36
Vibration : 4.73
Pression : 999.16
Alarme CloudWatch simulée pour Temperature
Alarme CloudWatch simulée pour Vibration
Alarme CloudWatch simulée pour Pressure

Temperature anomalies:
   sensor_id                  timestamp  temperature  vibration  pressure
3   SENSOR_1 2024-11-27 19:42:55.851725        31.58       3.29    990.07
4   SENSOR_1 2024-11-27 19:42:56.864705        32.02       1.06    985.36
6   SENSOR_1 2024-11-27 19:42:58.892111        34.21       2.47   1015.32
8   SENSOR_1 2024-11-27 19:43:00.927911        34.18       3.74    999.09
9   SENSOR_1 2024-11-27 19:43:01.941897        31.47       3.71   1003.75
10  SENSOR_1 2024-11-27 19:43:02.954723        30.53       1.38   1014.42
16  SENSOR_1 2024-11-27 19:43:09.036508        32.41       3.67   1008.24
24  SENSOR_1 2024-11-27 19:43:17.166212        32.01       1.04   1017.30
26  SENSOR_1 2024


ALERTE ! L'anomalie a été détectée. L'alarme se déclenche...


test_authenticate_user (__main__.TestIoTSensor) ... ok
test_collect_data (__main__.TestIoTSensor) ... 


La période d'alarme est terminée.

--- Exécution des Tests Unitaires ---
Bucket iot-sensor-data simulé
Bucket iot-sensor-data simulé


ok
test_detect_anomalies (__main__.TestIoTSensor) ... ok
test_generate_data (__main__.TestIoTSensor) ... ok
test_generate_report (__main__.TestIoTSensor) ... ok

----------------------------------------------------------------------
Ran 5 tests in 2.074s

OK


Données uploadées simulées : sensor_data_20241127_194458.json
Métriques publiées (simulation)
Dernières valeurs :
Température : 28.87
Vibration : 3.05
Pression : 1016.92
Alarme CloudWatch simulée pour Temperature
Alarme CloudWatch simulée pour Vibration
Alarme CloudWatch simulée pour Pressure
Bucket iot-sensor-data simulé

Tous les tests ont réussi !
